In [ ]:
import os
import sys
import rasterio
import subprocess
import stat
import shutil
import re
import time

import numpy as np
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt

from osgeo import gdal, gdalconst
from urllib.request import urlopen
from datetime import datetime
from scipy import ndimage
from scipy.stats import linregress

# MongoDB Database
"""from pymongo import MongoClient
client = MongoClient()

database = client.Satelites
db = database.Landsat"""



class Landsat:
    
    
    """Clase para trabajar con las Landsat de la nueva Collection 2 Level 2 del USGS"""
    
    def __init__(self, ruta_escena):
        
        """Inicializa un objeto Landsat basado en el path de la escena.

        Args:
            ruta_escena (str): Ruta al directorio de la escena Landsat.
        """
        
        # Definimos los paths de entrada
        self.ruta_escena = ruta_escena
        self.escena = os.path.split(self.ruta_escena)[1]
        self.ori = os.path.split(self.ruta_escena)[0]
        self.base = os.path.split(self.ori)[0]
        
        self.data = os.path.join(self.base, 'data')
        self.geo = os.path.join(self.base, 'geo')
        self.rad = os.path.join(self.base, 'rad')
        #self.nor = os.path.join(self.base, 'nor')
        self.pro = os.path.join(self.base, 'pro')
        
        # Definimos un grupo de variables de la escena
        data_escena = self.escena.split('_')
        self.sat = "L" + data_escena[0][-1]
        sensores = {'L4': 'TM', 'L5': 'TM', 'L7': 'ETM+', 'L8': 'OLI', 'L9': 'OLI'}
        self.sensor = sensores[self.sat]
        self.nprocesado = data_escena[1]
        self.path = data_escena[2][:3]
        self.row = data_escena[2][-3:]
        self.escena_date = data_escena[3]
        self.escena_procesado_date = data_escena[4]
        self.collection = data_escena[5]
        self.tier = data_escena[6]

        # Mascara de nuebes. Hay que hacerlo así porque sabe dios por qué ^!·/&"! los valores no son los mismos en OLI que en ETM+ y TM
        if self.sensor == 'OLI':
            self.cloud_mask_values = [21824, 21952]
        else:
            self.cloud_mask_values = [5440, 5504]

        #Definimos nombre last
        self.last_name = (self.escena_date + self.sat + self.sensor + self.path + '_' + self.row[1:]).lower()

        # Definimos paths de salida
        self.pro_escena = os.path.join(self.pro, self.last_name)
        os.makedirs(self.pro_escena, exist_ok=True)

        self.geo_escena = os.path.join(self.geo, self.last_name)
        os.makedirs(self.geo_escena, exist_ok=True)

        self.rad_escena = os.path.join(self.rad, self.last_name)
        os.makedirs(self.rad_escena, exist_ok=True)

        #self.nor_escena = os.path.join(self.nor, self.last_name)
        #os.makedirs(self.nor_escena, exist_ok=True)

        self.pro_escena = os.path.join(self.pro, self.last_name)
        os.makedirs(self.pro_escena, exist_ok=True)

        # Definimos máscaras a utilizar
        #self.equilibrado = os.path.join(self.data, 'Equilibrada.tif')
        #self.noequilibrado = os.path.join(self.data, 'NoEquilibrada.tif')
        #self.parametrosnor = {}
        #self.iter = 1

        #Creamos un diccionario a partir del MTL
        self.mtl = {}
        # Alumnos
        
        #Creamos las variables con la franja del espectro de cada banda
        if self.sat in ['L8', 'L9']:
            
            for i in os.listdir(self.ruta_escena):
                if i.endswith('.TIF'):
                    banda = os.path.splitext(i)[0].split('_')[-1]
            
                    if banda == 'B2':
                        self.blue = os.path.join(self.ruta_escena, i)
                    elif banda == 'B3':
                        self.green = os.path.join(self.ruta_escena, i)
                    elif banda == 'B4':
                        self.red = os.path.join(self.ruta_escena, i)
                    elif banda == 'B5':
                        self.nir = os.path.join(self.ruta_escena, i)
                    elif banda == 'B6':
                        self.swir1 = os.path.join(self.ruta_escena, i)
                    elif banda == 'B7':
                        self.swir2 = os.path.join(self.ruta_escena, i)
                    elif i.endswith('QA_PIXEL.TIF'):
                        self.qa = os.path.join(self.ruta_escena, i)
                        print(self.qa)
                    else:
                        continue
                
        elif self.sat in ['L7', 'L5']:
            
            for i in os.listdir(self.ruta_escena):
                if i.endswith('.TIF'):
                    banda = os.path.splitext(i)[0].split('_')[-1]
            
                    if banda == 'B1':
                        self.blue = os.path.join(self.ruta_escena, i)
                    elif banda == 'B2':
                        self.green = os.path.join(self.ruta_escena, i)
                    elif banda == 'B3':
                        self.red = os.path.join(self.ruta_escena, i)
                    elif banda == 'B4':
                        self.nir = os.path.join(self.ruta_escena, i)
                    elif banda == 'B5':
                        self.swir1 = os.path.join(self.ruta_escena, i)
                    elif banda == 'B7':
                        self.swir2 = os.path.join(self.ruta_escena, i)
                    elif i.endswith('QA_PIXEL.TIF'):
                        self.qa = os.path.join(self.ruta_escena, i)
                        print(self.qa)
                    else:
                        continue
        
        else:
            print('No encuentro ninguna escena Landsat')
            
        
        # Descargamos el quicklook de la escena 
        url_base = 'https://landsatlook.usgs.gov/gen-browse?size=rrb&type=refl&product_id={}'.format(self.mtl['LANDSAT_PRODUCT_ID'].strip('""'))
        self.qk_name = os.path.join(self.ruta_escena, self.escena + '_Quicklook.jpeg')
        
        if not os.path.exists(self.qk_name):
            qk = open(self.qk_name, 'wb')
            qk_open = urlopen(url_base)
            urlimg = qk_open.read()
            qk.write(urlimg)
            qk.close()

            print('QuicKlook descargado')
            
        else:
            print('El Quicklook ya estaba previamente descargado')
        
        #print(url_base)
        print('Landsat iniciada con éxito') 

        self.pn_cover = None
        #Creamos el json para instarlo en la base de datos MongoDB
        self.newesc = {'_id': self.last_name, 
                       'usgs_id': self.mtl['LANDSAT_SCENE_ID'][1:-1], 
                       'tier_id': self.mtl['LANDSAT_PRODUCT_ID'][1:-1],
                           'lpgs': self.mtl['PROCESSING_SOFTWARE_VERSION'][1:-1],
                       'category': self.mtl['COLLECTION_CATEGORY'][1:-1],
                       'Clouds': {'cloud_scene': float(self.mtl['CLOUD_COVER']),
                                 'land cloud cover': float(self.mtl['CLOUD_COVER_LAND'])},
                           'Info': {'Tecnico': 'LAST-EBD Auto', 
                                    'Iniciada': datetime.now(), 'Pasos': {'rad': '', 'nor': ''}}}

        """try:

            db.insert_one(self.newesc)

        except Exception as e:

            db.update_one({'_id': self.last_name}, {'$set':{'Info.Iniciada': datetime.now()}}, upsert=True)
            #print("Unexpected error:", type(e)) #se Podria dar un error por clave unica, por eso en
            #ese caso, lo que hacemos es actualizar la fecha en la que tratamos la imagen"""

        print('Landsat instanciada y (ahora no)subida a la base de datos')


 def get_hillshade(self):

        """Genera un hillshade de la escena usando parámetros solares (azimuth y elevación solar) 
        de los metadatos del MTL.txt.

        La idea es simular las sombras reales y quitarlas de los píxeles validos en la máscara de agua.
        """

        dtm = os.path.join(self.data, 'dtm_202_34.tif') #Por defecto esta en 29 y solo para la 202_34
        azimuth = None 
        elevation = None
        
        #Una vez tenemos estos parametros generamos el hillshade
        salida = os.path.join(self.pro_escena, 'hillshade.tif')
        cmd = ["gdaldem", "hillshade", "-az", "-alt", "-of", "GTIFF"]
        cmd.append(dtm)
        cmd.append(salida)
        cmd.insert(3, str(azimuth))
        cmd.insert(5, str(elevation))
        
        proc = subprocess.Popen(cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        stdout,stderr=proc.communicate()
        exit_code=proc.wait()

        if exit_code: 
            raise RuntimeError(stderr)
        else:
            print(stdout)
            print('Hillshade generado')
        
        
    def get_cloud_pn(self):

        """Calcula la cobertura de nubes dentro del Parque Nacional.

        Recorta la máscara de nubes (fmask) con un shapefile del Parque Nacional 
        para obtener la cobertura nubosa dentro de los límites del parque. 
        Actualiza la base de datos con los porcentajes de nubes (sobre tierra y escena completa MTL.txt) y sobre Doñana.
        """

        shape = os.path.join(self.data, 'Limites_PN_Donana.shp')
        crop = "-crop_to_cutline"

        for i in os.listdir(self.ruta_escena):
            if i.endswith('QA_PIXEL.TIF'):
                cloud = os.path.join(self.ruta_escena, i)

        #usamos Gdalwarp para realizar las mascaras, llamandolo desde el modulo subprocess
        cmd = ["gdalwarp", "-dstnodata" , "0" , "-cutline", ]
        path_masks = os.path.join(self.ruta_escena, 'masks')
        os.makedirs(path_masks, exist_ok=True)


        salida = os.path.join(path_masks, 'cloud_PN.TIF')
        cmd.insert(4, shape)
        cmd.insert(5, crop)
        cmd.insert(6, cloud)
        cmd.insert(7, salida)

        proc = subprocess.Popen(cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        stdout,stderr=proc.communicate()
        exit_code=proc.wait()

        if exit_code: 
            raise RuntimeError(stderr)


        ds = gdal.Open(salida)
        cloud = np.array(ds.GetRasterBand(1).ReadAsArray())
        
        mask = (cloud == self.cloud_mask_values[0]) | (cloud == self.cloud_mask_values[1])
        
        #################
        ##### DEFINIR LOS VALORES PARA LA MASCARA Y AÑADIR EL DATO A MONGO

        cloud_msk = cloud[mask]
        #print(cloud_msk.size)
        clouds = float(cloud_msk.size * 900)
        #print(clouds)
        PN = 533740500 
        self.pn_cover = round(100 - (clouds/PN) * 100, 2)
        ds = None
        cloud = None
        cloud_msk = None
        clouds = None        

        """try:

            db.update_one({'_id': self.last_name}, {'$set':{'Clouds.cloud_PN': self.pn_cover}},  upsert=True)

        except Exception as e:
            print("Unexpected error:", type(e), e)"""

        print("El porcentaje de nubes en el Parque Nacional es de " + str(self.pn_cover))


def remove_masks(self):

        """Elimina las máscaras temporales generadas durante el procesamiento de nubes.

        Borra los archivos de máscaras creados en el directorio 'masks' de la escena.
        """

        path_masks = os.path.join(self.ruta_escena, 'masks')
        for i in os.listdir(path_masks):

            name = os.path.join(path_masks, i)
            os.chmod(name, stat.S_IWRITE)
            os.remove(name)

        shutil.rmtree(path_masks)


    def projwin(self):

        """Aplica proyección y extensión geográfica a las bandas de la escena.
        """        
        
        olibands = {'B1': 'cblue_b1', 'B2': 'blue_b2', 'B3': 'green_b3', 'B4': 'red_b4', 'B5': 'nir_b5', 'B6': 'swir1_b6',
                   'B7': 'swir2_b7', 'PIXEL': 'fmask', 'B10': 'lst'}
        
        etmbands = {'B1': 'blue_b1', 'B2': 'green_b2', 'B3': 'red_b3', 'B4': 'nir_b4', 'B5': 'swir1_b5',
                   'B7': 'swir2_b7', 'PIXEL': 'fmask', 'B6': 'lst'}
        
        
        #geo = '/media/diego/31F8C0B3792FC3B6/EBD/Protocolo_v2_2024/geo'
        #path_rad = os.path.join(self.geo, self.escena)
        #os.makedirs(path_rad, exist_ok=True)
        wrs = os.path.join(self.data, 'wrs_202034.shp')
        
        for i in os.listdir(self.ruta_escena):
            
            if self.sat in ['L8', 'L9']:
                if i.endswith('.TIF'):

                    banda = i.split('_')[-1][:-4]

                    if banda in olibands.keys():
                        ins = os.path.join(self.ruta_escena, i)

                        name = self.escena_date + self.sat + self.sensor + self.path + '_' + self.row[1:] + '_g2_' + olibands[banda] + '.tif'
                        out = os.path.join(self.geo_escena, name.lower())

                        cmd = "gdalwarp -ot Int32 -srcnodata 0 -dstnodata '-9999' -tr 30 30 -te 633570 4053510 851160 4249530 -tap -cutline {} {} {}".format(wrs, ins, out)
                        print(cmd)
                        os.system(cmd)

                    else:
                        continue
                        
            elif self.sat in ['L7', 'L5', 'L4']:
                if i.endswith('.TIF'):

                    banda = i.split('_')[-1][:-4]

                    if banda in etmbands.keys():
                        ins = os.path.join(self.ruta_escena, i)

                        name = self.escena_date + self.sat + self.sensor + self.path + '_' + self.row[1:] + '_g2_' + etmbands[banda] + '.tif'
                        out = os.path.join(self.geo_escena, name.lower())

                        cmd = "gdalwarp -ot Int32 -srcnodata 0 -dstnodata '-9999' -tr 30 30 -te 633570 4053510 851160 4249530 -tap -cutline {} {} {}".format(wrs, ins, out)
                        print(cmd)
                        os.system(cmd)

                    else:
                        continue
                
                
            else:
                print('Lo siento, pero no encuentro el satélite')
                
                
    def coef_sr_st(self):

        """Aplica los coeficientes de reflectancia superficial y temperatura a las bandas.

        Esta función toma las bandas proyectadas y ajusta los valores para que sean
        interpretables en términos de reflectancia superficial y temperatura.
        """
    
        #path_geo = os.path.join(self.geo, self.escena)
        #path_rad = os.path.join(self.rad, self.escena)
        #os.makedirs(path_rad, exist_ok=True)
        
        for i in os.listdir(self.geo_escena):
    
            if i.endswith('.tif'):
    
                banda = i.split('_')[-1][:-4]
                
    
                if banda not in ['fmask', 'lst']:

                    print("Aplicando coeficientes a banda", banda)
                    
                    #nombre de salida que reemplace la  _g2_ del nombre original por _gr2_
                    rs = os.path.join(self.geo_escena, i)
                    out = os.path.join(self.rad_escena, i.replace('_g2_', '_gr2_'))
    
                    with rasterio.open(rs) as src:
                        RS = src.read(1)
                        meta = src.meta
    
                    # Aplicar coeficientes de reflectancia
                    sr = RS * 0.0000275 - 0.2
    
                    # Ajustar los valores al rango 0-1
                    sr = np.clip(sr, 0, 1)
                    
                    # Mantener los valores NoData
                    sr = np.where(RS == -9999, -9999, sr)
                    
                    meta.update(dtype=rasterio.float32)
    
                    with rasterio.open(out, 'w', **meta) as dst:
                        dst.write(sr.astype(rasterio.float32), 1)
    
                elif banda == 'lst':

                    print("Aplicando coeficientes a banda", banda)
                
                    #nombre de salida que reemplace la  _g2_ del nombre original por _gr2_
                    rs = os.path.join(self.geo_escena, i)
                    out = os.path.join(self.pro_escena, i.replace('_g2_', '_'))
    
                    with rasterio.open(rs) as src:
                        RS = src.read(1)
                        meta = src.meta
    
                    # Aplicar coeficientes de temperatura
                    lst = RS * 0.00341802 + 149.0
                    lst -= 273.15
    
                    # Mantener los valores NoData
                    lst = np.where(RS == -9999, -9999, lst)
                    
                    meta.update(dtype=rasterio.float32)
    
                    with rasterio.open(out, 'w', **meta) as dst:
                        dst.write(lst.astype(rasterio.float32), 1)
    
                elif banda == 'fmask':

                    print("Copiando", banda)
                    src = rs = os.path.join(self.geo_escena, i)
                    dst = os.path.join(self.rad_escena, i.replace('_g2_', '_'))                
                    shutil.copy(src, dst)
    
                else:                                       
                    continue
    
        print('Coeficientes aplicados con éxito')



In [ ]:
import os
import shutil
import re
import time
import subprocess
import pandas
import rasterio
import sys
import urllib
import fiona
import sqlite3
import math
import pymongo
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.features import geometry_mask
from osgeo import gdal, gdalconst
from datetime import datetime, date


from pymongo import MongoClient
client = MongoClient()

database = client.Satelites
db = database.Landsat

class Product(object):
    
    
    '''Esta clase genera los productos de inundacion, turbidez del agua y ndvi de las escenas normalizadas'''
    
        
    def __init__(self, ruta_nor):
        
        """Inicializa un objeto Product con la ruta de la escena normalizada.

        Args:
            ruta_nor (str): Ruta al directorio de la escena normalizada.
        """

        self.escena = os.path.split(ruta_nor)[1]
        self.raiz = os.path.split(os.path.split(ruta_nor)[0])[0]
        print(self.raiz)
        self.ori = os.path.join(self.raiz, 'ori')
        self.pro = os.path.join(self.raiz, 'pro')
        self.nor = ruta_nor
        self.productos = os.path.join(self.raiz, 'pro')
        self.data = os.path.join(self.raiz, 'data')
        self.water_masks = os.path.join(self.data, 'water_mask_pv2')
        self.temp = os.path.join(self.raiz, 'temp')
        os.makedirs(self.temp, exist_ok=True)
        
        self.ori_escena = os.path.join(self.raiz, os.path.join('ori', self.escena))
        self.nor_escena = ruta_nor
        self.pro_escena = os.path.join(self.pro, self.escena)
        os.makedirs(self.pro_escena, exist_ok=True)

        self.ndvi_escena = None
        self.ndwi_escena = None
        self.mndwi_escena = None
        self.flood_escena = None
        self.turbidity_escena = None
        self.depth_escena = None

        # Shape con recintos
        self.recintos = os.path.join(self.data, 'Recintos_Marisma.shp')
        # Salida con la superficie inundada por recinto
        #self.superficie_inundada = os.path.join(self.pro_escena, 'superficie_inundada.csv')
        

        # Tenemos que definir el sensor para coger los valores adecuados de Fmask
        if 'oli' in self.escena:
            self.sensor = 'OLI'
        elif 'etm' in self.escena:
            self.sensor = 'ETM+'
        elif 'l5tm' in self.escena:
            self.sensor = 'TM'
        else:
            print('what f* kind of satellite are you trying to work with?')

        # Mascara de nuebes. Hay que hacerlo así porque sabe dios por qué ^!·/&"! los valores no son los mismos en OLI que en ETM+ y TM
        if self.sensor == 'OLI':
            self.cloud_mask_values = [21824, 21952]
        else:
            self.cloud_mask_values = [5440, 5504]

        for i in os.listdir(self.nor_escena):
            if re.search('tif$', i):
                # Verificamos si el archivo es 'fmask' o 'hillshade'
                if 'fmask' in i:
                    self.fmask = os.path.join(self.nor, i)
                elif 'hillshade' in i:
                    self.hillshade = os.path.join(self.nor, i)
                else:
                    # Dividimos el nombre del archivo en partes
                    partes = i.split('_')
                    # Comprobamos si tiene al menos 3 partes (para evitar el error de índice)
                    if len(partes) >= 3:
                        banda = partes[-2]
                        if banda == 'blue':
                            self.blue = os.path.join(self.nor, i)
                        elif banda == 'green':
                            self.green = os.path.join(self.nor, i)
                        elif banda == 'red':
                            self.red = os.path.join(self.nor, i)
                        elif banda == 'nir':
                            self.nir = os.path.join(self.nor, i)
                        elif banda == 'swir1':
                            self.swir1 = os.path.join(self.nor, i)
                        elif banda == 'swir2':
                            self.swir2 = os.path.join(self.nor, i)


    def ndvi(self):

        """Calcula el NDVI (Índice de Vegetación de Diferencia Normalizada) para la escena.

        El NDVI se guarda como un archivo GeoTIFF y se actualiza en la base de datos.
        """

    def mndwi(self):

        """Calcula el MNDWI (Índice de Agua de Diferencia Normalizada) para la escena.

        El NDWI se guarda como un archivo GeoTIFF y se actualiza en la base de datos.
        """

    def flood(self):
        
        """Genera la máscara de inundación utilizando diversos criterios (e.g., NDWI, MNDWI, Slope).

        La máscara de inundación se guarda como un archivo GeoTIFF y se actualiza en la base de datos.
        Aplica reglas para determinar qué áreas son consideradas inundadas.
        """
    
        